In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found

In [2]:
#creating new csv 
import pandas as pd

df = pd.read_csv('/kaggle/input/csv-gm8k/gm8k.csv')
df.head()

,question,answer,final_solution
0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,18
1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,3
2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...,70000
3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,540
4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t...",20


In [3]:
#df.drop('final_solution',axis= 1 , inplace= True)

In [4]:
#df.head()
#df.to_csv('gm8k_ans.csv', index=False)

In [5]:
'''df = pd.read_csv('G:/Python/fine_tune_improve_mathematical_reasoning/Qwen2.5-1.5B_Fine-tuning/dataset/gm8k_new.csv')
df.head()'''

"df = pd.read_csv('G:/Python/fine_tune_improve_mathematical_reasoning/Qwen2.5-1.5B_Fine-tuning/dataset/gm8k_new.csv')\ndf.head()"

In [3]:
from datasets import Dataset
dataset = Dataset.from_pandas(df[["question", "answer", "final_solution"]])
dataset = dataset.train_test_split(test_size= 0.2)

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import bitsandbytes as bnb  

model_id = "Qwen/Qwen2.5-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,           
    device_map="cuda",         
    trust_remote_code=True,
)
#model = model.to("cuda")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [9]:
def preprocess_qwen(example):
    full_prompt = f"<|user|>\nSolve the following math problem step-by-step:\n{example['question']}\n<|assistant|>\n{example['answer']}"
    encoded = tokenizer(full_prompt, truncation=True, padding="max_length", max_length=1024)
    encoded["labels"] = encoded["input_ids"].copy()
    encoded["question"] = example["question"]
    encoded["answer"] = example["answer"]
    encoded["final_solution"] = example["final_solution"]
    return encoded


In [11]:
tokenized_dataset = dataset.map(preprocess_qwen, batched=False)


Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

Map:   0%|          | 0/264 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

model_id = "Qwen/Qwen2.5-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto",     
    trust_remote_code=True,
    torch_dtype=torch.float16   
)


model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,                          
    lora_alpha=32,               
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.1,                  
    bias="none",                     
    task_type="CAUSAL_LM"    
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

print(f"Model on device: {next(model.parameters()).device}")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 2,179,072 || all params: 1,545,893,376 || trainable%: 0.1410
Model on device: cuda:0


In [12]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


training_args = TrainingArguments(
    output_dir="./qwen-lora-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    fp16=True, 
    logging_steps=50,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    report_to="none", 
    learning_rate=2e-4,
    warmup_steps=100,
    optim="paged_adamw_8bit",
    label_names=["labels"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,

)


In [13]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,1.253300
100,0.884100
150,0.784300


TrainOutput(global_step=198, training_loss=0.9277083079020182, metrics={'train_runtime': 4300.778, 'train_samples_per_second': 0.736, 'train_steps_per_second': 0.046, 'total_flos': 2.55229426925568e+16, 'train_loss': 0.9277083079020182, 'epoch': 3.0})

In [16]:
model.save_pretrained("/kaggle/working/qwen_new-lora-adapters")

In [19]:
tokenizer.save_pretrained("/kaggle/working/qwen_new-lora-adapters")


('/kaggle/working/qwen_new-lora-adapters/tokenizer_config.json',
 '/kaggle/working/qwen_new-lora-adapters/special_tokens_map.json',
 '/kaggle/working/qwen_new-lora-adapters/vocab.json',
 '/kaggle/working/qwen_new-lora-adapters/merges.txt',
 '/kaggle/working/qwen_new-lora-adapters/added_tokens.json',
 '/kaggle/working/qwen_new-lora-adapters/tokenizer.json')

In [21]:
import os

output_path = "/kaggle/working/qwen_new-lora-adapters"
print("Exists:", os.path.exists(output_path))
print("Contents:", os.listdir(output_path) if os.path.exists(output_path) else "Not found")


Exists: True
Contents: ['added_tokens.json', 'special_tokens_map.json', 'adapter_model.safetensors', 'tokenizer.json', 'adapter_config.json', 'vocab.json', 'tokenizer_config.json', 'merges.txt', 'README.md']


In [22]:
import shutil
shutil.make_archive("/kaggle/working/qwen_new-lora-adapters", 'zip', "/kaggle/working/qwen_new-lora-adapters")


'/kaggle/working/qwen_new-lora-adapters.zip'